In [1]:
import numpy as np
import healpy as hp
import time
from numpy import linalg as LA
from subprocess import Popen, run
from scipy import integrate
from astropy import constants as const
from astropy import units as u
from scipy.special import spence
from scipy.signal import argrelextrema

import matplotlib.pyplot as plt
from nbodykit.lab import BigFileCatalog
import warnings
warnings.filterwarnings('ignore')
from nbodykit.transform import ConcatenateSources, CartesianToEquatorial

from colossus.cosmology import cosmology
cosmo = cosmology.setCosmology('planck18')
from colossus.halo import concentration


In [2]:
"""
import sys
argvs = sys.argv
print(argvs)
i_min = int(argvs[1]);
i_max = int(argvs[2]);
m_min = float(argvs[3]);
m_max = float(argvs[4]);
"""

# range of scale factor (a[i_min] to a[i_max])
i_min = 40; i_max = 41; 

# range of mass (10^14 to 10^20)
m_min = 14; m_max = 20; 

### Bins of scale factor a 
edges = np.array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])
    

In [3]:
z_edges = 1./edges - 1.
zbin = (z_edges[1:] + z_edges[:-1])/2.
Nzbin = len(zbin)
Nmbin = 50

rf_list = np.zeros((Nzbin, Nmbin))
rho_list = np.zeros((Nzbin, Nmbin))
p_list = np.zeros((Nzbin, Nmbin))

rf_list = np.loadtxt("data/rf_list.dat")
rho0_list = np.loadtxt("data/rho0_list.dat")
P0_list = np.loadtxt("data/P0_list.dat")

Nc_z = np.int32(np.loadtxt("data/Nc_list.dat"))
Mmin_z = np.loadtxt("data/Mmin_list.dat")
Mmax_z = np.loadtxt("data/Mmax_list.dat")

In [4]:
Nside = 2048; Npix = hp.nside2npix(Nside);
fout = 0; 
ksz = 0; # 0:tsz, 1:ksz

if(ksz==0):
    ymap = np.zeros(Npix)
    outdir = "../data/13_25_nside%d_tsz_proper/" %(Nside)
elif(ksz==1):
    kmap = np.zeros(Npix)
    outdir = "../data/13_25_nside%d_ksz_proper/" %(Nside)
else:
    print("tsz or ksz?")

cmd1 = ["mkdir", "-p",  outdir]
Popen(cmd1)

In [5]:
# Path to simulations

# At NERSC
#basedir = "/global/cfs/cdirs/m3035/yici/rfof_proc262144_nc8192_size5000_nsteps60lin_ldr0_rcvtrue_fstnone_pnf2_lnf2_s100_pgdtrue_dhf1.0000_tiled0.20_fll_elllim_10000_npix_4096/"
# At IPMU
basedir = "/gpfs02/work/tanimura/data/sims/halfdome/"

halodir = basedir + 'usmesh/'
lightcone1 = BigFileCatalog(halodir,dataset='RFOF')

In [6]:
# Constant
GG = const.G.to('kpc3 / (M_sun s2)').value # kpc3 / (s2 Msun)
fb = cosmo.Ob0/cosmo.Om0
cc = const.c.to('kpc / s').value
rhoc0 = cosmo.rho_c(0) * cosmo.h**2 # Msun h2/kpc3 -> Msun/kpc3

In [7]:
# Function to read simulations

def read_range(cat, amin, amax):
    """ Read a portion of the lightcone between two red shift ranges
        The lightcone from FastPM is sorted in Aemit and an index is built.
        So we make use of that.
        CrowCanyon is z > 0; We paste the mirror image to form a full sky.
    """
    edges = cat.attrs['aemitIndex.edges']
    offsets = cat.attrs['aemitIndex.offset']
    start, end = edges.searchsorted([amin, amax])
    if cat.comm.rank == 0:
        cat.logger.info("Range of index is %d to %d" %(( start + 1, end + 1)))
    start = offsets[start + 1]
    end = offsets[end + 1]
    cat =  cat.query_range(start, end)
    
    cat['Mass'] = cat['Length'] * cat.attrs['M0'] * 1e10 # [Msun/h]
    
    if cat.csize > 0:
        cat['RA'], cat['DEC'] = CartesianToEquatorial(cat['Position'], frame='galactic')
    else:
        cat['RA'] = 0
        cat['DEC'] = 0
    return cat


In [8]:
# Functions for rf_gas, rho(r), P(r) 
# (Osato et al 2023: Baryon pasting algorithm: halo-based and particle-based pasting methods)

# Table 1
fstar = 0.026
Sstar = 0.12
gamma = 1.2
nn = 1./(gamma - 1.)
alpha_nt = 0.18
beta_nt = 0.5
n_nt = 0.8
fmax = 4**(-n_nt)/alpha_nt

def phi_func(rr):
    return np.interp(rr, r_kpc, phi_s)

# eq(10)
def theta_func(rr, rho0, P0):
    return 1. + (gamma-1)/gamma * (rho0/P0) *  (phi0 - phi_func(rr)) 

# eq(8)
def Ptot_func(rr, rho0, P0):
    return P0 * theta_func(rr, rho0, P0)**(nn+1) 

# eq(9)
def rhog_func(rr, rho0, P0):
    return rho0 * theta_func(rr, rho0, P0)**(nn) 

# eq(19)
def Rnt(rr, redshift, R500):
    fz = np.min([ (1.+redshift)**beta_nt, (fmax-1)*np.tanh(beta_nt*redshift)+1. ]) # eq(20)
    return alpha_nt * fz * (rr/R500)**(n_nt)

# Thermal pressure to electron pressure (Osato et al 2023 eq(24))
XX = 0.76
th2e = (2*XX+2)/(5*XX+3)

def Pe(rr, redshift, rho0, P0, R500):
    Pth = Ptot_func(rr, rho0, P0) * np.max([0, 1 - Rnt(rr, redshift, R500)])    
    return Pth * th2e


In [9]:
# 3D -> 2D tSZ profile (integral along LOS)

sigt = const.sigma_T.to('kpc2').value
me = const.m_e.to('M_sun').value

def p3d(r_3d, redshift, Rmax, rho0, P0, R500): # 3D model function
    return Pe(r_3d, redshift, rho0, P0, R500)

def y2d_mpc(r_2d, r_3d, redshift, Rmax, rho0, P0, R500):
    return 2*p3d(r_3d, redshift, Rmax, rho0, P0, R500)*r_3d/np.sqrt(r_3d**2 - r_2d**2) 

def Ycyl_mpc(r_2d, redshift, Rmax, rho0, P0, R500): # projected on 2D 
    if (r_2d < Rmax):
        Ycyl = sigt/(me*cc**2) * integrate.quad(lambda r_3d: y2d_mpc(r_2d, r_3d, redshift, Rmax, rho0, P0, R500), r_2d, Rmax)[0] 
        return Ycyl
    else: 
        return 0

def Ycyl_theta(theta_arcmin, redshift, Rmax, rho0, P0, R500): # projected on 2D 

    DC_kpc = cosmo.comovingDistance(0, redshift)/cosmo.h * 1e3 # Mpc/h -> kpc
    x_kpc = np.deg2rad(theta_arcmin/60.) * DC_kpc  # deg -> rad -> kpc

    Ycyl = Ycyl_mpc(x_kpc, redshift, Rmax, rho0, P0, R500)
    return Ycyl


In [10]:
# electron density at z = 0
Ob0 = cosmo.Ob0
ue = 1.148 
mp = const.m_p.to('Msun').value
kpc2km = const.kpc.to('km').value
ne0 = (rhoc0*Ob0)/(ue*mp) # [1/kpc3]  # mean electron density at z=0


In [11]:
# 3D -> 2D kSZ profile (integral along LOS)

def Ne(rr, redshift, rho0, P0, R500):

    ne_z = rhog_func(rr, rho0, P0) /(ue*mp) # [1/kpc3] mean ne at z
    return ne_z 

def n3d(r_3d, redshift, Rmax, rho0, P0, R500): # 3D model function
    return Ne(r_3d, redshift, rho0, P0, R500)

def k2d_mpc(r_2d, r_3d, redshift, Rmax, rho0, P0, R500):
    return 2*n3d(r_3d, redshift, Rmax, rho0, P0, R500)*r_3d/np.sqrt(r_3d**2 - r_2d**2) 

def Kcyl_mpc(r_2d, redshift, Rmax, rho0, P0, R500, Vz): # projected on 2D 
    if (r_2d <= Rmax):
        Kcyl = sigt * (Vz/cc) * integrate.quad(lambda r_3d: k2d_mpc(r_2d, r_3d, redshift, Rmax, rho0, P0, R500), r_2d, Rmax)[0]
        return Kcyl
    else: 
        return 0

def Kcyl_theta(theta_arcmin, redshift, Rmax, rho0, P0, R500, Vz): # projected on 2D 

    DC_kpc = cosmo.comovingDistance(0, redshift)/cosmo.h * 1e3 # Mpc/h -> kpc
    x_kpc = np.deg2rad(theta_arcmin/60.) * DC_kpc  # deg -> rad -> kpc

    Kcyl = Kcyl_mpc(x_kpc, redshift, Rmax, rho0, P0, R500, Vz)
    return Kcyl


In [12]:
r_kpc = 10**np.arange(0,4,0.02) # kpc/h -> kpc

start = time.time()
#for I in range(20,100):
for I in range(i_min,i_max):

    if(Nc_z[I] == 0):
        continue;

    else:
        mbin_z = np.logspace(np.log10(Mmin_z[I]), np.log10(Mmax_z[I]), Nmbin)
        
        ### Get rf, rho0, P0 for Mvir, z
        # remove -1
        ind1_i = (rho0_list[I]!=-1) # -1 happens for rf, rho0, P0
        mbin_z_i1 = mbin_z[ind1_i==1]
        rf_z_i1 = rf_list[I][ind1_i==1]    
        rho0_z_i1 = rho0_list[I][ind1_i==1]
        P0_z_i1 = P0_list[I][ind1_i==1]    
        
        ### remove local peaks
        ind2_i_all = np.arange(len(rho0_z_i1))
        
        # rf
        ind2_rf_hpeaks_i = argrelextrema(rf_z_i1, np.greater, order=1)[0]
        ind2_rf_lpeaks_i = argrelextrema(rf_z_i1, np.less, order=1)[0]
        ind2_rf_i = np.delete(ind2_i_all, np.append(ind2_rf_hpeaks_i, ind2_rf_lpeaks_i))        
        # rho0
        ind2_rho0_hpeaks_i = argrelextrema(rho0_z_i1, np.greater, order=1)[0]
        ind2_rho0_lpeaks_i = argrelextrema(rho0_z_i1, np.less, order=1)[0]
        ind2_rho0_i = np.delete(ind2_i_all, np.append(ind2_rho0_hpeaks_i, ind2_rho0_lpeaks_i))

        # P0
        ind2_P0_hpeaks_i = argrelextrema(P0_z_i1, np.greater, order=1)[0]
        ind2_P0_lpeaks_i = argrelextrema(P0_z_i1, np.less, order=1)[0]
        ind2_P0_i = np.delete(ind2_i_all, np.append(ind2_P0_hpeaks_i, ind2_P0_lpeaks_i))        
        
        mbin_z_i_rf=mbin_z_i1[ind2_rf_i]; rf_z_i = rf_z_i1[ind2_rf_i];
        mbin_z_i_rho0=mbin_z_i1[ind2_rho0_i]; rho0_z_i = rho0_z_i1[ind2_rho0_i];
        mbin_z_i_P0=mbin_z_i1[ind2_P0_i]; P0_z_i = P0_z_i1[ind2_P0_i];        

        #plt.loglog(mbin_z_i_P0, P0_z_i)

        # Halos 
        Omz = cosmo.Om(zbin[I])
        vv = 18*np.pi**2 + 82*(Omz-1.) - 39*(Omz-1.)**2
        rhoc = cosmo.rho_c(zbin[I]) * cosmo.h**2 # Msun h2/kpc3 -> Msun/kpc3

        lc1 = read_range(lightcone1,edges[I],edges[I+1])
        Mass_all = lc1['Mass'].compute()/cosmo.h # [Msun/h] -> [Msun]
        RA_all = lc1['RA'].compute()
        Dec_all = lc1['DEC'].compute()

        #cond_i = (1e13 < Mass_all) * (Mass_all <= 1e14)
        print("m_min, m_max", m_min, m_max)
        cond_i = (10**m_min < Mass_all) * (Mass_all <= 10**m_max)

        Nc = np.sum(cond_i)
        if(Nc==0):
            print("Nc:", Nc)        
            exit()

        Mass = Mass_all[cond_i==1]
        RA = RA_all[cond_i==1]        
        Dec = Dec_all[cond_i==1]

        ### Only for kSZ ###
        if(ksz==1):
            def tau_func(redshift):
                return sigt * cc * ((1.+redshift)**2) * ne0 /(cosmo.Hz(zbin[I])/kpc2km/1e3) 

            Pos_all = lc1['Position'].compute()
            Vel_all = lc1['Velocity'].compute()
            Dis_all = LA.norm(Pos_all,axis=1)
            Pos_all_unit_vec = Pos_all/Dis_all.reshape(len(Pos_all),1)
            Vr_all = np.sum(Vel_all *  Pos_all_unit_vec, axis=1)/kpc2km # [km/s] -> [kpc/s]
            tau_z = integrate.quad(tau_func, 0, zbin[I])[0] 
            Vr = Vr_all[cond_i==1]
        ### 

        theta = 0.5 * np.pi - np.deg2rad(Dec) #from declination to theta
        phi = np.deg2rad(RA) #from right ascension to phi
        
        DC_kpc_i = cosmo.comovingDistance(0, zbin[I])/cosmo.h * 1e3 # Mpc/h -> kpc

        print("Nc:", Nc)        
        print("%d th redshift" %(I))

        ### Paint SZ signal of each halo
        for J in range(Nc):
        #for J in range(1):    
        #for J in [115]:    
        
            if(J%10000 == 0):
                middle = time.time()
                print("%d th redshift" %(I))
                print("%d th out of %d" %(J, Nc))
                print("total time = %f (min)" %( (middle - start)/60.))
        
            # halo
            Mvir = Mass[J]
            #print("Mvir, z:", np.log10(Mvir), zbin[I])
            rvir = (3*Mvir/(4*np.pi*vv*rhoc))**(1./3.) # kpc
            cvir = concentration.concentration(Mvir*cosmo.h, 'vir', zbin[I], model = 'ishiyama21')

            # Osato et al 2023
            s_vir = r_kpc/rvir # eq(3)
            gc = 1./(np.log(1 + cvir) - cvir /(1 + cvir)) # eq(4)
            cs_vir = cvir*s_vir

            rho_s = rhoc * ( vv * cvir**2 * gc / (3 * s_vir * (1+cs_vir)**2) )
            Mv = 4 * np.pi * (rvir**3) * vv * rhoc / 3.
            M_s = Mv * gc * (np.log(1+cs_vir) - cs_vir/(1+cs_vir))
            Vv = np.sqrt( GG * Mv / rvir)  # km/s -> kpc/s
            
            phi_s = - Vv**2 * gc * np.log(1 + cs_vir)/s_vir # km2 / s2
            phi0 = - cvir * gc * Vv**2 # km2 / s2

            Li2 = spence(1+cs_vir)
            sigv_r = np.sqrt(Vv**2 * 0.5 * cvir**2 * gc * s_vir * (1. + cs_vir)**2 * ( np.pi**2 - np.log(cs_vir) - 1./cs_vir - 1./(1.+cs_vir)**2  - 6./(1.+cs_vir) + (1. + 1./cs_vir**2 - 4./cs_vir - 2./(1+cs_vir)) * np.log(1.+cs_vir) + 3*(np.log(1+cs_vir))**2 + 6 * Li2) )

            # rf, rho0, P0
            rf_j = np.interp(np.log10(Mvir), np.log10(mbin_z_i_rf), rf_z_i)
            rho0_j = np.interp(np.log10(Mvir),np.log10(mbin_z_i_rho0),rho0_z_i)
            P0_j = np.interp(np.log10(Mvir), np.log10(mbin_z_i_P0), P0_z_i)

            # y
            #print("rf, rho0, P0:", rf_j, rho0_j, P0_j)
            rout = rf_j
            theta_radian_max_j = rout/DC_kpc_i
            
            vec_j = hp.ang2vec(theta[J], phi[J])
            pix_num_j = hp.query_disc(Nside, vec_j, theta_radian_max_j, inclusive=True)
            Npix_j = len(pix_num_j)
            #print("Npix around a halo:", Npix_j)

            if(Npix_j==0):
                continue;

            else:
                pix_vec_j = hp.pix2vec(Nside, pix_num_j)
                dtheta_radian_j = np.arccos(np.dot(vec_j, pix_vec_j))
                dtheta_arcmin_j = np.rad2deg(dtheta_radian_j) * 60.
                for k in range(Npix_j):

                    if(ksz==1):
                        k_k = Kcyl_theta(dtheta_arcmin_j[k], zbin[I], rout, rho0_j, P0_j, rvir, Vr[J]) * np.exp(-tau_z)

                        if( np.isfinite(k_k)==True):
                            kmap[pix_num_j[k]] = kmap[pix_num_j[k]] + k_k
                            
                    else:
                        y_k = Ycyl_theta(dtheta_arcmin_j[k], zbin[I], rout, rho0_j, P0_j, rvir)
                        
                        if( np.isfinite(y_k)==True):
                            ymap[pix_num_j[k]] = ymap[pix_num_j[k]] + y_k



if(fout==1):            
    if(ksz==1):
        hp.write_map(outdir + "kmap_paint_nside%d_imin%d_imax%d_mmin%.1f_mmax%.1f.fits" %(Nside, i_min, i_max, m_min, m_max), kmap, overwrite=True)
    else:
        hp.write_map(outdir + "ymap_paint_nside%d_imin%d_imax%d_mmin%.1f_mmax%.1f.fits" %(Nside, i_min, i_max, m_min, m_max), ymap, overwrite=True)




m_min, m_max 14 20
Nc: 18497
40 th redshift
40 th redshift
0 th out of 18497
total time = 1.676741 (min)
40 th redshift
10000 th out of 18497
total time = 1.964975 (min)
